<a href="https://colab.research.google.com/github/mykcs/seg-keras-AyushThakur-0404/blob/main/seg_keras_AyushThakur_0404_colab05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports and Setups

In [1]:
!pip install -qq wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.5 MB/s eta 0:00:00


In [2]:
import wandb
from wandb.keras import WandbMetricsLogger
from wandb.keras import WandbEvalCallback

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models

import tensorflow_datasets as tfds

import os
import numpy as np
from argparse import Namespace
import matplotlib.pyplot as plt

In [4]:
!nvidia-smi

Wed Apr  3 18:50:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
configs = Namespace(
    epochs = 10,
    img_size = 128,
    batch_size = 64,
    num_classes = 3,
)

configs

Namespace(epochs=10, img_size=128, batch_size=64, num_classes=3)

# Dataloader

We will be using Oxford Pets Dataset which we can directly get from TensorFlow Datasets.

In [6]:
train_ds, valid_ds = tfds.load('oxford_iiit_pet', split=["train", "test"])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/3680 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_iiit_pet/3.2.0.incomplete89K547/oxford_iiit_pet-train.tfrecord*...:…

Generating test examples...:   0%|          | 0/3669 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_iiit_pet/3.2.0.incomplete89K547/oxford_iiit_pet-test.tfrecord*...: …

Dataset oxford_iiit_pet downloaded and prepared to /root/tensorflow_datasets/oxford_iiit_pet/3.2.0. Subsequent calls will reuse this data.


In [7]:
AUTOTUNE = tf.data.experimental.AUTOTUNE


def parse_data(example):
    # Parse image
    image = example["image"]
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, size=(configs.img_size, configs.img_size))

    # Parse mask
    mask = example["segmentation_mask"] - 1 # ground truth labels are [1,2,3].
    mask = tf.image.resize(mask, size=(configs.img_size, configs.img_size), method='nearest')
    mask = tf.one_hot(tf.squeeze(mask, axis=-1), depth=configs.num_classes)

    return image, mask

trainloader = (
    train_ds
    .shuffle(1024)
    .map(parse_data, num_parallel_calls=AUTOTUNE)
    .batch(configs.batch_size)
    .prefetch(AUTOTUNE)
)

validloader = (
    valid_ds
    .map(parse_data, num_parallel_calls=AUTOTUNE)
    .batch(configs.batch_size)
    .prefetch(AUTOTUNE)
)

## Model

In [8]:
# ref: https://github.com/ayulockin/deepimageinpainting/blob/master/Image_Inpainting_Autoencoder_Decoder_v2_0.ipynb
class SegmentationModel:
    '''
    Build UNET based model for segmentation task.
    '''
    def prepare_model(self, OUTPUT_CHANNEL, input_size=(configs.img_size, configs.img_size, 3)):
        inputs = layers.Input(input_size)

        conv1, pool1 = self.__ConvBlock(32, (3,3), (2,2), 'relu', 'same', inputs)
        conv2, pool2 = self.__ConvBlock(64, (3,3), (2,2), 'relu', 'same', pool1)
        conv3, pool3 = self.__ConvBlock(128, (3,3), (2,2), 'relu', 'same', pool2)
        conv4, pool4 = self.__ConvBlock(256, (3,3), (2,2), 'relu', 'same', pool3)

        conv5, up6 = self.__UpConvBlock(512, 256, (3,3), (2,2), (2,2), 'relu', 'same', pool4, conv4)
        conv6, up7 = self.__UpConvBlock(256, 128, (3,3), (2,2), (2,2), 'relu', 'same', up6, conv3)
        conv7, up8 = self.__UpConvBlock(128, 64, (3,3), (2,2), (2,2), 'relu', 'same', up7, conv2)
        conv8, up9 = self.__UpConvBlock(64, 32, (3,3), (2,2), (2,2), 'relu', 'same', up8, conv1)

        conv9 = self.__ConvBlock(32, (3,3), (2,2), 'relu', 'same', up9, False)

        outputs = layers.Conv2D(OUTPUT_CHANNEL, (3, 3), activation='softmax', padding='same')(conv9)

        return models.Model(inputs=[inputs], outputs=[outputs])

    def __ConvBlock(self, filters, kernel_size, pool_size, activation, padding, connecting_layer, pool_layer=True):
        conv = layers.Conv2D(filters=filters, kernel_size=kernel_size, activation=activation, padding=padding)(connecting_layer)
        conv = layers.Conv2D(filters=filters, kernel_size=kernel_size, activation=activation, padding=padding)(conv)
        if pool_layer:
          pool = layers.MaxPooling2D(pool_size)(conv)
          return conv, pool
        else:
          return conv

    def __UpConvBlock(self, filters, up_filters, kernel_size, up_kernel, up_stride, activation, padding, connecting_layer, shared_layer):
        conv = layers.Conv2D(filters=filters, kernel_size=kernel_size, activation=activation, padding=padding)(connecting_layer)
        conv = layers.Conv2D(filters=filters, kernel_size=kernel_size, activation=activation, padding=padding)(conv)
        up = layers.Conv2DTranspose(filters=up_filters, kernel_size=up_kernel, strides=up_stride, padding=padding)(conv)
        up = layers.concatenate([up, shared_layer], axis=3)

        return conv, up

#### Initialize Model and Compile

In [9]:
# output channel is 3 because we have three classes in our mask
tf.keras.backend.clear_session()
model = SegmentationModel().prepare_model(configs.num_classes)

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 32)         896       ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 128, 128, 32)         9248      ['conv2d[0][0]']              
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 64, 64, 32)           0         ['conv2d_1[0][0]']            
 D)                                                                                           

## Callback

In [10]:
segmentation_classes = ['pet', 'pet_outline', 'background']

# returns a dictionary of labels
def labels():
    l = {}
    for i, label in enumerate(segmentation_classes):
        l[i] = label
    return l

In [11]:
class WandbSemanticLogger(WandbEvalCallback):
    def __init__(
        self,
        validloader,
        data_table_columns=["index", "image"],
        pred_table_columns=["epoch", "index", "image", "prediction"],
        num_samples=100,
    ):
        super().__init__(
            data_table_columns,
            pred_table_columns,
        )

        self.val_data = validloader.unbatch().take(num_samples)

    def add_ground_truth(self, logs):
        for idx, (image, mask) in enumerate(self.val_data):
            self.data_table.add_data(
                idx,
                self._prepare_wandb_mask(
                    image.numpy(),
                    np.argmax(mask.numpy(), axis=-1),
                    "ground_truth"
                )
            )

    def add_model_predictions(self, epoch, logs):
        data_table_ref = self.data_table_ref
        table_idxs = data_table_ref.get_index()

        for idx, (image, mask) in enumerate(self.val_data):
            prediction = self.model.predict(tf.expand_dims(image, axis=0), verbose=0)
            prediction = np.argmax(tf.squeeze(prediction, axis=0).numpy(), axis=-1)

            self.pred_table.add_data(
                epoch,
                data_table_ref.data[idx][0],
                self._prepare_wandb_mask(
                    data_table_ref.data[idx][1],
                    np.argmax(mask.numpy(), axis=-1),
                    "ground_truth"
                ),
                self._prepare_wandb_mask(
                    data_table_ref.data[idx][1],
                    prediction,
                    "prediction"
                )
            )

    def _prepare_wandb_mask(self, image, mask, mask_type):
        return wandb.Image(
            image,
            masks = {
                "ground_truth": {
                    "mask_data": mask,
                    "class_labels": labels()
            }})

## Train

In [13]:
run = wandb.init(project='seg-keras-AyushThakur-0404',
                 name = "colab05",
                 group="colab",
                 job_type="first-try",
                 config=configs
                 )


wandb: Currently logged in as: mykcs (team-mykcs). Use `wandb login --relogin` to force relogin


In [14]:
_ = model.fit(
    trainloader,
    epochs=configs.epochs,  # 10
    validation_data=validloader,
    callbacks=[
        WandbMetricsLogger(log_freq=2),
        WandbSemanticLogger(validloader)
      ]
    )


wandb:   202 of 202 files downloaded.  


Epoch 1/10
 6/58 [==>...........................] - ETA: 14s - loss: 1.0595

58/58 [==============================] - 76s 847ms/step - loss: 0.8921 - val_loss: 0.8270
Epoch 2/10
58/58 [==============================] - 48s 802ms/step - loss: 0.7210 - val_loss: 0.6842
Epoch 3/10
58/58 [==============================] - 42s 646ms/step - loss: 0.6666 - val_loss: 0.6378
Epoch 4/10
58/58 [==============================] - 41s 653ms/step - loss: 0.6128 - val_loss: 0.5870
Epoch 5/10
58/58 [==============================] - 40s 652ms/step - loss: 0.5542 - val_loss: 0.5480
Epoch 6/10
58/58 [==============================] - 43s 644ms/step - loss: 0.5197 - val_loss: 0.4825
Epoch 7/10
58/58 [==============================] - 39s 635ms/step - loss: 0.4577 - val_loss: 0.4499
Epoch 8/10
58/58 [==============================] - 41s 652ms/step - loss: 0.4288 - val_loss: 0.5360
Epoch 9/10
58/58 [==============================] - 39s 636ms/step - loss: 0.4145 - val_loss: 0.4103
Epoch 10/10
58/58 [==============================] - 52s 830ms/step - loss: 0.3770 - val_loss: 0.3846


In [15]:
run.finish()

batch/batch_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▇▇▇▅▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▁▂▂▂▂▂▂▁▁▁▁▁
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▄▃▃▂▂▂▁
epoch/val_loss,█▆▅▄▄▃▂▃▁▁
batch/batch_step,578
batch/learning_rate,0.001
batch/loss,0.3771
epoch/epoch,9
